In [50]:
using DataFrames, DataFramesMeta, LinearAlgebra, CSV, Distributions, Optim, Revise, Plots

In [51]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule

In [52]:
# I = 100;              # Number of consumers
J = 3;                  # Number of firms
K = 1;                  # Product characteristics
nT = 5;                 # Number of markets
# β = [1, -1];            # Preferences
β = -1.;            # Preferences
varζ = 3;               # Variance of the random taste
# rangeJ = [J, J];        # TODO: set a min and max firms per market
rangeB = [100, 1000];       # consumers per market (e.g. census block)
# varX = 2;               # Variance of X
varξ = 1;               # Variance of xi

In [53]:
dist_thresh = 0.5
df = BLPmodule.build_dist_data(J, β, nT, rangeB, varζ, varξ, dist_thresh)

,s,s0,q,q0,j,dist,B,t,q_j
,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Float64,Float64
1,0.254509,0.274042,127.0,136.747,1,0.0742573,499,1.0,375.297
2,0.228768,0.274042,114.155,136.747,2,0.17096,499,1.0,474.561
3,0.242681,0.274042,121.098,136.747,3,0.137927,499,1.0,481.206
4,0.274926,0.272186,48.6619,48.177,1,0.0268394,177,3.0,375.297
5,0.24416,0.272186,43.2164,48.177,2,0.218378,177,3.0,474.561
6,0.208727,0.272186,36.9447,48.177,3,0.185344,177,3.0,481.206
7,0.324399,0.386521,103.808,123.687,2,0.325542,320,4.0,474.561
8,0.28908,0.386521,92.5057,123.687,3,0.358575,320,4.0,481.206
9,0.227634,0.26605,199.635,233.326,1,0.158063,877,5.0,375.297


In [54]:
# Simulate
# df = BLPmodule.simulate_data(I, J, β, nT, rangeJ, rangeB, varζ, varX, varξ)
# Retrieve data
T = Int.(df.t)
X_ = df.dist
# s_ = df.s
q_j_ = df.q_j
s0_ = df.s0
q0_ = df.q0
# IV_ = [df.x1 df.w]
# Z_ = [df.x1 df.z1]
ζ_ = [rand(Normal(0,1), 1, I); zeros(K, I)]; # Draw shocks (less)

In [55]:
# using BLPmodule
# Minimize GMM objective function
# varζ_ = optimize(x -> BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, x[1])[1], [3.0], LBFGS()).minimizer[1];

In [56]:
agg_q = [q_j_[df.j .== j][1] for j in 1:J]
total_outside = sum(unique(df.q0))
agg_q0 = repeat([total_outside], J)
total_B = sum([sum(agg_q), total_outside])
agg_shares = agg_q ./ total_B
agg_s0 = agg_q0 ./ total_B;

In [70]:
agg_q

3-element Vector{Float64}:
 375.2967868281019
 474.56135958130574
 481.20568471194986

In [57]:
agg_shares

3-element Vector{Float64}:
 0.20037201645921082
 0.25336965273961864
 0.2569170767282167

In [58]:
agg_s0

3-element Vector{Float64}:
 0.2893412540729539
 0.2893412540729539
 0.2893412540729539

In [69]:
X_

11-element Vector{Float64}:
 0.07425725620262957
 0.17096009434286352
 0.1379266015322187
 0.026839367280010462
 0.21837798326548263
 0.1853444904548378
 0.3255415455552728
 0.35857503836591764
 0.15806297745465514
 0.08715437309083796
 0.05412088028019313

In [67]:
t1ind = df.t .== 1;

In [68]:
using BLPmodule
deltas = BLPmodule.compute_delta(agg_shares, agg_s0, X_, ζ_, T);

BoundsError: BoundsError: attempt to access 3-element Vector{Float64} at index [Bool[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [61]:
δ0 = zeros(1, size(ζ_, 2))
shares = BLPmodule.implied_shares(X_[t1ind,:], ζ_, deltas, δ0)
println("computed shares:       ", shares)
println("true aggregate shares: ", agg_shares)

UndefVarError: UndefVarError: t1ind not defined

In [62]:
# δ0 = zeros(1, size(ζ_, 2))
# for t in unique(T)
#     tind = df.t.==t
#     shares = BLPmodule.implied_shares(X_[tind,:], ζ_, deltas1, δ0)
#     println("computed shares: ", shares)
# end
# println(agg_shares)

In [63]:
# β_blp = BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, varζ_)[2];

In [64]:
# print("Estimated BLP coefficients: $β_blp")

In [65]:
# # Compute logit estimate
# y = log.(df.s) - log.(df.s0)
# β_logit = inv(IV_' * X_) * (IV_' * y)
# print("Estimated logit coefficients: $β_logit")